In [4]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error

In [12]:
def apply_depolarizing_noise(qc, qubits, error_prob):
    """
    Apply depolarizing noise to specified qubits in the quantum circuit.
    :param qc: QuantumCircuit object
    :param qubits: List of qubits to apply noise on
    :param error_prob: Probability of depolarizing noise
    """
    noise_model = NoiseModel()
    dep_error_1q = depolarizing_error(error_prob, 1)  # 1-qubit depolarizing error
    dep_error_2q = depolarizing_error(error_prob, 2)  # 2-qubit depolarizing error
    
    for q in qubits:
        noise_model.add_quantum_error(dep_error_1q, ["u1", "u2", "u3", "id"], [q])
    
    for q1, q2 in [(0, 1), (0, 2)]:  # Apply 2-qubit depolarizing error to CX gates
        noise_model.add_quantum_error(dep_error_2q, ["cx"], [q1, q2])
    
    return noise_model

In [9]:
# Define a simple QEC circuit with depolarizing noise
def create_qec_circuit(noise_prob=0.01):
    qc = QuantumCircuit(3, 3)  # 3 qubits, 3 classical bits
    
    # Encoding step (simplified)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(0, 2)
    
    # Introduce depolarizing noise
    noise_model = apply_depolarizing_noise(qc, [0, 1, 2], noise_prob)
    
    # Syndrome measurement
    qc.cx(0, 1)
    qc.cx(0, 2)
    qc.measure([0, 1, 2], [0, 1, 2])
    
    return qc, noise_model

In [7]:
# Simulate the QEC circuit with depolarizing noise
def simulate_qec_with_noise(noise_prob=0.01, shots=1024):
    qc, noise_model = create_qec_circuit(noise_prob)
    simulator = AerSimulator()
    result = simulator.run(qc, noise_model=noise_model, shots=shots).result()
    counts = result.get_counts()
    return counts

In [13]:
# Run the simulation
if __name__ == "__main__":
    noise_probability = 0.05  # Example noise probability
    results = simulate_qec_with_noise(noise_probability)
    print("Measurement Outcomes:", results)

Measurement Outcomes: {'101': 21, '100': 12, '110': 18, '001': 463, '011': 21, '111': 15, '000': 446, '010': 28}
